In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wq_df=pd.read_csv('/kaggle/input/water-potability/water_potability.csv')

In [ ]:
wq_df.info()

### Check for missing values

In [ ]:
import missingno as msno

#msno.matrix(wq_df)

sns.heatmap(wq_df.isnull(), cbar=False)

In [ ]:
msno.matrix(wq_df)

### Below columns have missing values:
  -  ph
  -  Sulfate
  -  Trihalomethanes

In [ ]:
wq_df.describe()

In [ ]:
#### Univariate analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
for col in wq_df.columns:
    if col not in ['Potability']:
        sns.boxplot(data=wq_df,y=col)
        #sns.barplot(data=wq_df,y=col)
        plt.show()
    

In [ ]:
wq_df.head()

### Lets remove outliers

In [ ]:
for col in wq_df.columns:
    if col not in ['Potability']:
        wq_df=wq_df[wq_df[col] < wq_df[col].quantile(0.999)]
        wq_df=wq_df[wq_df[col] > wq_df[col].quantile(0.01)]


In [ ]:
wq_df.Potability.value_counts()

In [ ]:
for col in wq_df.columns:
    if col not in ['Potability']:
        fig,ax =plt.subplots(1,3, figsize=(13,5))
        sns.set_color_codes("muted")
        sns.set(style="whitegrid")
        sns.boxplot(data=wq_df,y=col,x='Potability',ax=ax[0])
        sns.barplot(data=wq_df,y=col,x='Potability',ax=ax[1])
        sns.violinplot(data=wq_df,y=col,x='Potability',ax=ax[2])
            
               
    #plt.show()
        plt.show()
    

#### There is not much variation between potability distribution

#### Check Potability distribution percentage

In [ ]:
wq_df.groupby("Potability")['Potability'].count().plot.pie(autopct="%.1f%%",labels=['red','white'])

### Bivariate analysis

#### Check correlation among varaibles

In [ ]:
plt.figure(figsize =(10,10))
sns.heatmap(wq_df.corr(),annot=True)
plt.show()

#### We see some corelation between
  - ph and Hardness
  - Sulfate and Solids

#### lets check the relation among all the variables using pair plot

In [ ]:
sns.pairplot(wq_df)

#### From above analysis we see all the varaibles are independent

#### check relation among ph , Hardness and Potability

In [ ]:
sns.lmplot("ph", "Hardness", data=wq_df, hue="Potability", fit_reg=False)


#### check relation Sulfate , Solids and Potability

In [ ]:
sns.lmplot("Sulfate", "Solids", data=wq_df, hue="Potability", fit_reg=False)

In [ ]:
#sns.lmplot("ph", "Hardness", data=wq_df, hue="Potability", fit_reg=False)
sns.lineplot(data=wq_df,y="ph",x="Hardness")

In [ ]:
sns.lineplot(data=wq_df,y="Sulfate",x="Solids")

### Fill missing values

- ph
- sulfate
- Trihalomethanes

In [ ]:
columns_with_missing_vls=['ph','Sulfate','Trihalomethanes']
for col in columns_with_missing_vls:
    wq_df[col] = wq_df[col].fillna(wq_df[col].mean())

#### Check missing values again

In [ ]:
wq_df.info()

#### Lets apply standard scaler 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
wq_df[wq_df.columns.difference(['Potability'])]

In [ ]:
wq_df_scaler=scaler.fit_transform(wq_df[wq_df.columns.difference(['Potability'])])

In [ ]:
wq_df_scaler_df=pd.DataFrame(wq_df_scaler,columns=[wq_df[wq_df.columns.difference(['Potability'])].columns])

In [ ]:
wq_df_scaler_df

#### Test and train split

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
Y=wq_df.pop('Potability')
X=wq_df_scaler_df

In [ ]:
X

### Split test and train with equal proportion

In [ ]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(k_neighbors=4)

X,Y=sm.fit_resample(X,Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###  Model building with XGBhhost

In [ ]:
model = XGBClassifier(radom_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy: %.2f%%" % (accuracy_score(y_pred,y_test) * 100.0))

#### The accuracy with XGBhhost is around ~ 70 %

### Model builidng with RandomForest classfier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=42)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
y_pred_rc = rfc.predict(X_test)

In [ ]:
print("Accuracy: %.2f%%" % (accuracy_score(y_pred_rc,y_test) * 100.0))

### As per above analysis, The best accuracy is ~ 73 with Random Forest.